# 在链上使用记忆
- LLMChain
- ConversationChain
- 自定义
- 同一个链合并使用多个记忆
- 给一个多参数链增加记忆
_____

## LLMChain

In [10]:
from langchain_ollama import OllamaLLM
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

ollama_llm = OllamaLLM(model="llama3.1:8b", temperature=0)

# 自定义模板
template = """你是一个可以和人类对话的机器人。
{chat_history}
人类：{human_input}
机器人："""

prompt = PromptTemplate(
  template=template,
  input_variables=["chat_history", "human_input"],
)

memory = ConversationBufferMemory(
  memory_key="chat_history",
)

chain = LLMChain(
  llm=ollama_llm,
  memory=memory,
  prompt=prompt,
  verbose=True
)

In [13]:
# chain.predict(human_input="你好")
# chain.predict(human_input="今天成都的天气真好，已经很久没有出太阳了")
chain.predict(human_input="成都今天的天气怎么样")




> Entering new LLMChain chain...
Prompt after formatting:
你是一个可以和人类对话的机器人。
Human: 你好
AI: 你好！很高兴见到你。今天过得怎么样？
Human: 今天成都的天气真好，已经很久没有出太阳了
AI: 太好了！成都的天气确实是美丽的。太阳出来了，肯定会让大家的心情更好一些。有去外面晒晒太阳吗？
人类：成都今天的天气怎么样
机器人：

> Finished chain.


'很不错啊！刚刚你也说了，已经很久没有出太阳了，所以今天的天气应该是比较晴朗的。'

## ChatModel

In [5]:
from langchain_ollama.chat_models import ChatOllama
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

### ChatPromptTemplate 具有特定的格式

In [6]:
prompt = ChatPromptTemplate.from_messages(
  [
    SystemMessage(content="你好，我是一个可以和人类对话的机器人"),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{human_input}"),
  ]
)

print(prompt.format(human_input="你好", chat_history=[]))

System: 你好，我是一个可以和人类对话的机器人
Human: 你好


In [11]:
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
chat_llm = ChatOllama(model="llama3.1:8b", temperature=0)

from langchain.chains.llm import LLMChain
chain = LLMChain(
  llm=chat_llm,
  memory=memory,
  prompt=prompt,
  verbose=True
)

In [15]:
# chain.predict(human_input="你好")
# chain.predict(human_input="今天成都的天气真好，已经很久没有出太阳了")
chain.predict(human_input="成都今天的天气怎么样")



> Entering new LLMChain chain...
Prompt after formatting:
System: 你好，我是一个可以和人类对话的机器人
Human: 你好
AI: 很高兴见到你！我是你的机器人朋友。有什么想聊天或问的问题吗？
Human: 你好
AI: 再次打招呼！今天过得怎么样呢？有任何新鲜事要分享吗？
Human: 今天成都的天气真好，已经很久没有出太阳了
AI: 太好了！阳光总是能让人心情更好。希望你在外面散步时能感受到温暖的阳光。成都的天气变化确实比较多，有时候会突然变冷或变热，但总体来说还是很美丽的城市呢。
Human: 你好
AI: 再次打招呼！看起来你还在享受着今天的好天气呢？有没有计划去哪里玩耍或者游览一下成都的景点？
Human: 今天成都的天气真好，已经很久没有出太阳了
AI: 太好了！阳光总是能让人心情更好。希望你在外面散步时能感受到温暖的阳光。成都的天气变化确实比较多，有时候会突然变冷或变热，但总体来说还是很美丽的城市呢。

你喜欢哪个季节的成都天气？春天的花季，夏天的炎热，秋天的红叶，还是冬天的雾霭？
Human: 成都今天的天气怎么样
AI: 根据你的描述，我知道了！成都今天的天气很好，已经很久没有出太阳了。所以，今天应该是比较晴朗和温暖的天气呢！
Human: 成都今天的天气怎么样

> Finished chain.


'我可以通过网络获取一些信息，但是如果你想知道今天成都的具体天气情况（如温度、湿度等），我建议你可以查一下当地的天气预报网站或APP。'

## ConversationChain
提示词模板就是 `input` `history`

In [21]:
from langchain.chains.conversation.base import ConversationChain
from langchain_ollama.llms import OllamaLLM
from langchain.memory import ConversationBufferMemory

ollama_llm = OllamaLLM(model="llama3.1:8b", temperature=0)
memory = ConversationBufferMemory(memory_key="history",return_messages=True)
chain = ConversationChain(
  llm=ollama_llm,
  memory=memory,
  verbose=True,
)

In [25]:
# chain.predict(input="你好")
# chain.predict(input="今天成都的天气真好，已经很久没有出太阳了")
chain.predict(input="成都今天的天气怎么样")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！我很高兴和您交谈。您想问些什么呢？我可以提供有关各种话题的信息，包括但不限于天文学、历史、科学、艺术等。或者，您也可以与我讨论一些更轻松的话题，如电影、音乐或最新的科技新闻。', additional_kwargs={}, response_metadata={}), HumanMessage(content='今天成都的天气真好，已经很久没有出太阳了', additional_kwargs={}, response_metadata={}), AIMessage(content="[AIMessage(content='您说的对！根据我最近的数据更新，成都的天气确实是非常舒适。根据中国气象局的预报，今天的成都将有大量的阳光和蓝天。事实上，成都的平均日照时数已经超过了8个小时，这意味着您可以在外面度过一个美好的日子。', additional_kwargs={}, response_metadata={})]", additional_kwargs={}, response_metadata={})]
Human: 成都今天的天气怎么样
AI:

> Finished chain.


'根据我最近的数据更新，成都的天气确实是非常舒适。根据中国气象局的预报，今天的成都将有大量的阳光和蓝天。事实上，成都的平均日照时数已经超过了8个小时，这意味着您可以在外面度过一个美好的日子。'

##### 针对COnversation做一个定制

In [27]:
# 自定义一下，对其进行覆盖
from langchain.prompts import PromptTemplate

template = """下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。
当前对话：
{history}
human:{input}
AI:"""

prompt = PromptTemplate(template=template,
                        input_variables=["history","input"])
memory = ConversationBufferMemory(ai_prefix="AI",return_messages=True)
chain = ConversationChain(
  llm=ollama_llm,
  memory=memory,
  prompt=prompt, # 这里覆盖了ConversationChain的预制Prompt
  verbose=True,
)

In [34]:
# chain.predict(input="你好")
# chain.predict(input="今天成都的天气真好，已经很久没有出太阳了")
# chain.predict(input="成都今天的天气怎么样")
# chain.predict(input="成都明天的天气怎么样")
chain.predict(input="我叫什么名字")




> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。
当前对话：
[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='你好！很高兴和你交流。有什么问题需要我的帮助吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='今天成都的天气真好，已经很久没有出太阳了', additional_kwargs={}, response_metadata={}), AIMessage(content="[AIMessage(content='是啊，最近几天天气一直不太好呢！不过今天看起来确实是晴天了。', additional_kwargs={}, response_metadata={})]", additional_kwargs={}, response_metadata={}), HumanMessage(content='成都今天的天气怎么样', additional_kwargs={}, response_metadata={}), AIMessage(content='[AIMessage(content="根据目前的天气预报，成都今天的天气应该是多云转晴，温度在15-20度之间。虽然最近几天天气不太好，但今天看起来确实是晴天了。", additional_kwargs={}, response_metadata={})]', additional_kwargs={}, response_metadata={}), HumanMessage(content='成都明天的天气怎么样', additional_kwargs={}, response_metadata={}), AIMessage(content='根据目前的天气预报，成都明天的天气应该是多云转阴，温度在14-19度之间。虽然今天看起来很晴朗，但预计明天会有几场小雨。', ad

'人类姥爷，这个问题我不知道。'

______
## 同一个链合并使用多个memory

In [36]:
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ( ConversationSummaryMemory,
  ConversationBufferMemory,
  CombinedMemory # 合并Chain
  )
from langchain.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM

In [38]:
ollama_llm = ollama_llm

# 使用ConversationSummaryMemory对对话进行总结
summary = ConversationSummaryMemory(
  llm=ollama_llm,
  input_key="input"
)

cov_memory = ConversationBufferMemory(
  memory_key="history_new",
  input_key="input"
)

memory = CombinedMemory(
  memories=[summary, cov_memory]
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。
之前的对话摘要：
{history}
当前对话：
{history_new}
human:{input}
AI:"""

prompt = PromptTemplate(
  template=TEMPLATE,
  input_variables=["history","history_new","input"]
)

conv_chain = ConversationChain(
  llm=ollama_llm,
  memory=memory,
  prompt=prompt,
  verbose=True,
)

In [41]:
# conv_chain.run("你对加密市场是如何分析的？")
# conv_chain.run("这么厉害，你觉得量化投资的前景如何？")
conv_chain.run("这么厉害，你觉得加密市场的前景如何？")



> Entering new ConversationChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。
之前的对话摘要：
Here is the updated summary:

The human asks how the AI analyzes the cryptocurrency market. The AI's analysis is based on multiple data sources and algorithms, including monitoring global cryptocurrency trading volumes, price fluctuations, and related news events, as well as analyzing technical indicators such as MACD and RSI to determine market trends.

The AI also uses machine learning models to train on historical data and predict future trends and risks. These models are based on complex algorithms, including time series analysis and association rule mining.

Additionally, the AI references reports from authoritative media and research institutions in the cryptocurrency field to understand market dynamics and policy impacts. The AI's final report is a data-driven analysis that combines all this information for user reference.

The human th

'人类姥爷，这个问题我知道！根据我的分析和数据驱动的模型，目前加密市场呈现出快速增长和波动的特征。虽然有很多挑战和风险，但也存在着巨大的潜力和机会。\n\n例如，比特币价格已经突破了之前的历史高点，并且其他一些加密货币也表现出了强劲的增长势头。但是，市场也面临着监管、安全性和可持续性的问题，这些挑战可能会影响市场的前景。\n\n但是，如果你问我关于量化投资的前景，我还是会说：人类姥爷，这个问题我不知道。'

## 多参数链增加记忆

In [47]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("long_text.txt") as f:
  text = f.read()

# 文本切分
text_splitter = CharacterTextSplitter(
  separator='\n',
  chunk_size = 100,
  chunk_overlap = 5,
)
texts = text_splitter.split_text(text)

# 使用 Ollama 本地的Embedding
embedding = OllamaEmbeddings(model="nomic-embed-text:latest",temperature=0)

# 使用 Chroma 存储 向量
docsearch_vecstor = Chroma.from_texts(texts, embedding)

query = "百度有哪些产品？"
docs = docsearch_vecstor.similarity_search(query)


Created a chunk of size 155, which is longer than the specified 100
Created a chunk of size 126, which is longer than the specified 100
Created a chunk of size 164, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 182, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100
Created a chunk of size 164, which is longer than the specified 100
Created a chunk of size 381, which is longer than the specified 100
Created a chunk of size 127, which is longer than the specified 100
Created a chunk of size 126, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 133, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 138, which is longer tha

In [54]:
## 构架问答对话链

from langchain.chains.question_answering import load_qa_chain
from langchain_ollama.llms import OllamaLLM
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。参考一下相关文档以及历史对话信息，
AI会根据此组织和回答内容：：
{context}
{chat_history}
human:{human_input}
AI:"""

ollama_llm = ollama_llm
prompt = PromptTemplate(
  template=TEMPLATE,
  input_variables=["context", "chat_history", "human_input"]
)

# 使用ConversationBufferMemory对对话进行缓存
memory = ConversationBufferMemory(
  memory_key="chat_history",
  input_key="human_input",
  return_messages=True
)

# 加载对话链
chat_chain = load_qa_chain(
  llm=ollama_llm,
  memory=memory,
  prompt=prompt,
  verbose=True,
  chain_type="stuff"
)


chat_chain({"input_documents":docs, "human_input":"百度有哪些产品？"})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
下面是一段AI与人类的对话，AI会针对人类的提问，做尽可能详细的回答，
如果AI不知道答案，会直接回复：人类姥爷，这个问题我不知道。参考一下相关文档以及历史对话信息，
AI会根据此组织和回答内容：：
百度浏览器：做个有趣的人

百度知道：总有一个人知道你问题的答案

百度安全：有AI更安全

百度安全：有AI更安全
[]
human:百度有哪些产品？
AI:

> Finished chain.

> Finished chain.


{'input_documents': [Document(metadata={}, page_content='百度浏览器：做个有趣的人'),
  Document(metadata={}, page_content='百度知道：总有一个人知道你问题的答案'),
  Document(metadata={}, page_content='百度安全：有AI更安全'),
  Document(metadata={}, page_content='百度安全：有AI更安全')],
 'human_input': '百度有哪些产品？',
 'chat_history': [HumanMessage(content='百度有哪些产品？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好问题！百度是一个多元化的互联网公司，拥有许多不同的产品和服务。以下是其中一些主要的产品：\n\n1. **百度搜索**：这是百度最著名的产品之一，是一个全能的搜索引擎，可以帮助用户快速找到网上信息。\n2. **百度知道**：是一个问答式的知识分享平台，用户可以在这里提问和回答问题，获取各种知识和经验。\n3. **百度浏览器**：是百度推出的一个自定义浏览器，可以帮助用户更好地使用互联网，并提供一些额外的功能和服务。\n4. **百度安全**：是一个专门为用户提供网络安全保护的产品，包括防病毒、防垃圾邮件等功能。\n5. **百度云**：是一个云计算平台，提供存储、计算、数据库等服务，让用户可以更方便地管理和使用自己的数据。\n6. **百度地图**：是一个专业的地理信息系统，可以帮助用户查找路线、查看地图等。\n7. **百度推广**：是百度为企业提供的在线营销平台，包括搜索引擎优化（SEO）、付费推广等服务。\n\n这些产品和服务都是百度为了让用户更好地使用互联网而开发的。', additional_kwargs={}, response_metadata={})],
 'output_text': '好问题！百度是一个多元化的互联网公司，拥有许多不同的产品和服务。以下是其中一些主要的产品：\n\n1. **百度搜索**：这是百度最著名的产品之一，是一个全能的搜索引擎，可以帮助用户快速找到网上信息。\n


![memory——chain](pngs/主要的预制链和memory工具.png)
![本章小结](pngs/小节.png)